In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

from astropy.table import Table

In [30]:
def correct_gband(bp_rp, astrometric_params_solved, phot_g_mean_mag, phot_g_mean_flux):
    """
    code from https://github.com/agabrown/gaiaedr3-6p-gband-correction
    Correct the G-band fluxes and magnitudes for the input list of Gaia EDR3 data.
    
    Parameters
    ----------
    
    bp_rp: float, numpy.ndarray
        The (BP-RP) colour listed in the Gaia EDR3 archive.
    astrometric_params_solved: int, numpy.ndarray
        The astrometric solution type listed in the Gaia EDR3 archive.
    phot_g_mean_mag: float, numpy.ndarray
        The G-band magnitude as listed in the Gaia EDR3 archive.
    phot_g_mean_flux: float, numpy.ndarray
        The G-band flux as listed in the Gaia EDR3 archive.
        
    Returns
    -------
    
    The corrected G-band magnitudes and fluxes. The corrections are only applied to
    sources with a 2-paramater or 6-parameter astrometric solution fainter than G=13, 
    for which a (BP-RP) colour is available.
    
    Example
    -------
    
    gmag_corr, gflux_corr = correct_gband(bp_rp, astrometric_params_solved, phot_g_mean_mag, phot_g_mean_flux)
    """

    if np.isscalar(bp_rp) or np.isscalar(astrometric_params_solved) or np.isscalar(phot_g_mean_mag) or np.isscalar(phot_g_mean_flux):
        bp_rp = np.float64(bp_rp)
        astrometric_params_solved = np.int64(astrometric_params_solved)
        phot_g_mean_mag = np.float64(phot_g_mean_mag)
        phot_g_mean_flux = np.float64(phot_g_mean_flux)
    
    if not (bp_rp.shape == astrometric_params_solved.shape == phot_g_mean_mag.shape == phot_g_mean_flux.shape):
        raise ValueError('Function parameters must be of the same shape!')
        
    do_not_correct = np.isnan(bp_rp) | (phot_g_mean_mag<13) | (astrometric_params_solved == 31)
    bright_correct = np.logical_not(do_not_correct) & (phot_g_mean_mag>=13) & (phot_g_mean_mag<=16)
    faint_correct = np.logical_not(do_not_correct) & (phot_g_mean_mag>16)
    bp_rp_c = np.clip(bp_rp, 0.25, 3.0)
    
    correction_factor = np.ones_like(phot_g_mean_mag)
    correction_factor[faint_correct] = 1.00525 - 0.02323*bp_rp_c[faint_correct] + \
        0.01740*np.power(bp_rp_c[faint_correct],2) - 0.00253*np.power(bp_rp_c[faint_correct],3)
    correction_factor[bright_correct] = 1.00876 - 0.02540*bp_rp_c[bright_correct] + \
        0.01747*np.power(bp_rp_c[bright_correct],2) - 0.00277*np.power(bp_rp_c[bright_correct],3)
    
    gmag_corrected = phot_g_mean_mag - 2.5*np.log10(correction_factor)
    gflux_corrected = phot_g_mean_flux * correction_factor
    
    return gmag_corrected, gflux_corrected

In [3]:
pulsar_data = np.genfromtxt('ipta_3arcsec_wpm-result.csv', delimiter=',', names=True, dtype=None)

<ipython-input-3-ee42e60aa3b8>:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  pulsar_data = np.genfromtxt('ipta_3arcsec_wpm-result.csv', delimiter=',', names=True, dtype=None)


In [4]:
pulsar_data = Table(pulsar_data)
pulsar_data.sort(['names', 'angdist'])

In [5]:
df = pandas.DataFrame(data=pulsar_data['names'])
names_keep = ['J0437-4715', 'J1012+5307', 'J1024-0719', 'J1732-5049', 'J1747-4036', 'J1843-1113']#, 'J1955+2908']
pulsar_data = pulsar_data[np.isin((df['names'].str.decode("utf-8")), names_keep)] # keep only the sources we wrote about

In [6]:
df = pandas.DataFrame(data=pulsar_data['designation'])

objects_keep = ['Gaia EDR3 4789864076732331648', 'Gaia EDR3 851610861391010944', 'Gaia EDR3 3775277872387310208', 'Gaia EDR3 5946288492263176704', 'Gaia EDR3 5957827763757710080', 'Gaia EDR3 5957827763757708544', 'Gaia EDR3 4106823440438736384']
pulsar_data = pulsar_data[np.isin((df['designation'].str.decode("utf-8")), objects_keep)] # keep only the sources we wrote ab


In [16]:
gmag=pulsar_data['phot_g_mean_mag']

In [21]:
bprp=pulsar_data['bp_rp']

In [23]:
astro_sol=pulsar_data['astrometric_params_solved']

In [27]:
gflux=pulsar_data['phot_g_mean_flux']

In [29]:
corr=[]
nums=[0,1,2,3,4,5,6]
for ii in nums:
    print(correct_gband(bprp[ii], astro_sol[ii], gmag[ii], gflux[ii]))

(20.351167849142037, 136.29483003795514)
(19.586607, 275.6159095)
(19.148087245234883, 412.77377492091694)
(20.809143278403006, 89.3901086979164)
(20.147984, 164.3438848)
(19.77844122429924, 230.97791344801956)
(19.754900037840212, 236.04038533078784)


In [ ]:
#all so dim there is no change!